In [ ]:
import pandas as pd
import re

df = pd.read_csv('Amazon-Products.csv')
print(df.head())  # Display the first few rows

In [ ]:
# Define data cleaning functions
def clean_price(price):
    """Clean price field by removing currency symbols and commas, then convert to float."""
    if pd.isna(price):  # Check for NaN or None
        return None
    if isinstance(price, (int, float)):  # If already numeric, return as float
        return float(price)
    # Clean string price
    cleaned_price = price.replace('₹', '').replace(',', '').strip()
    return float(cleaned_price)  # Convert to float for handling decimals


def clean_no_of_ratings(no_of_ratings):
    """Clean the no_of_ratings field by handling normal values, NaN, and strings like 'Only 2 left in stock.'"""
    if pd.isna(no_of_ratings):  # Check for NaN or None
        return None
    
    # If the no_of_ratings is a string, handle possible commas and extract numbers using regex
    if isinstance(no_of_ratings, str):
        # Remove commas if any
        no_of_ratings = no_of_ratings.replace(',', '')
        
        # Check if the string contains a number (like 'Only 2 left in stock.' or '2,255')
        match = re.search(r'\d+', no_of_ratings)  # Look for one or more digits in the string
        if match:
            # Return the first matched number as an integer
            return int(match.group(0))
        
        # If no digits are found in the string, return None
        return None
    
    # If no_of_ratings is already a number, return it
    return no_of_ratings


def clean_ratings(ratings):
    """
    Clean the ratings field by handling NaN, float values in string,
    and non-numeric or invalid rating strings.
    """
    if pd.isna(ratings):  # Check for NaN or None
        return None
    
    # If the ratings is a string, handle possible float values
    if isinstance(ratings, str):
        # Remove commas if any
        ratings = ratings.replace(',', '').strip()
        
        # Check if the string contains a float value (e.g., '4.2', '3.5')
        match = re.search(r'^\d+(\.\d+)?$', ratings)  # Match integers or floats (e.g., '4', '4.2')
        if match:
            # Return the matched value as a float type
            return float(match.group(0))
        
        # If the string is not a valid rating, return None
        return None
    
    # If the ratings is already a float or int, return it as is
    return ratings


def extract_asin(link):
    """Extract ASIN from the product link."""
    return link.split('/dp/')[1].split('/')[0] if '/dp/' in link else None

def clean_image(image):
    """Clean the image field by removing the base URL and keeping only the file name."""
    if pd.isna(image):  # Check for NaN or None
        return None

    return re.sub(r'/images/W/IMAGERENDERING_[^/]+/', '/', image)

# Apply data cleaning functions to the DataFrame
def clean_dataframe(df):
    """Apply all cleaning functions to the DataFrame."""
    df['discount_price_clean'] = df['discount_price'].apply(clean_price)
    df['actual_price_clean'] = df['actual_price'].apply(clean_price)
    df['no_of_ratings_clean'] = df['no_of_ratings'].apply(clean_no_of_ratings)
    df['ratings'] = df['ratings'].apply(clean_ratings)
    df['image'] = df['image'].apply(clean_image)


    discount_nan_rows = df[df['discount_price_clean'].isna()]
    print(f"discount_nan_rows: {len(discount_nan_rows)}")
    actual_price_nan_rows = df[df['actual_price_clean'].isna()]
    print(f"actual_price_nan_rows: {len(actual_price_nan_rows)}")
    no_of_ratings_nan_rows = df[df['no_of_ratings_clean'].isna()]
    print(f"no_of_ratings_nan_rows: {len(no_of_ratings_nan_rows)}")
    ratings_nan_rows = df[df['ratings'].isna()]
    print(f"ratings_nan_rows: {len(ratings_nan_rows)}")


    # Drop rows where 'actual_price_clean' is NaN
    df = df.dropna(subset=['actual_price_clean'])
    df['actual_price'] = df['actual_price_clean']
    df['discount_price'] = df['discount_price_clean'].fillna(df['actual_price_clean'])
    df['no_of_ratings'] = df['no_of_ratings_clean'].fillna(0)
    df['ratings'] = df['ratings'].fillna(0)
    df.loc[df['no_of_ratings_clean'].isna(), 'ratings'] = 0

    
    # discount_nan_rows = df[df['discount_price'].isna()]
    # print(f"discount_nan_rows: {len(discount_nan_rows)}")
    # actual_price_nan_rows = df[df['actual_price'].isna()]
    # print(f"actual_price_nan_rows: {len(actual_price_nan_rows)}")
    # no_of_ratings_nan_rows = df[df['no_of_ratings'].isna()]
    # print(f"no_of_ratings_nan_rows: {len(no_of_ratings_nan_rows)}")
    # ratings_nan_rows = df[df['ratings'].isna()]
    # print(f"ratings_nan_rows: {len(ratings_nan_rows)}")
    

    
    # ratings_counts =  df['ratings'].value_counts(dropna=False)
    # print(ratings_counts)
    # actual_price_counts = df['actual_price'].value_counts(dropna=False)
    # print(actual_price_counts)
    # discount_price_counts = df['discount_price'].value_counts(dropna=False)
    # print(discount_price_counts)
    # no_of_ratings_counts = df['no_of_ratings'].value_counts(dropna=False)
    # print(no_of_ratings_counts)


    
    df['asin'] = df['link'].apply(extract_asin)

    df = df.drop(columns=['no_of_ratings_clean', 'actual_price_clean', 'discount_price_clean'])
    return df

# Process the DataFrame
df = clean_dataframe(df)

# Print the number of rows and columns
print(f"The DataFrame has {df.shape[0]} rows and {df.shape[1]} columns.")

# df.to_csv('products.csv', index=False)
# print("DataFrame has been saved to 'products.csv'.")

# compress the csv to zip folder
output_path = '../src/seed/seed_data.zip'
compression_options = dict(method='zip', archive_name=f'products.csv')
df.to_csv(output_path, index=False, compression=compression_options)
print(f"DataFrame has been saved and compressed to '{output_path}'")